In [13]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"


In [14]:
#import sklearn as sk
import transformers
#import sentence-transformers
from  datasets import Dataset 
import torch 
import pandas as pd
from transformers import Trainer, TrainingArguments

#from tensorflow import keras

In [15]:



df = pd.read_csv("/home/profniggastein/PycharmProjects/ReducedDataset/gcj2009_balanced_exp.csv")

df = df[["flines", "solution"]].dropna()
df = df.rename(columns={"flines": "code", "solution": "label"})
df["label"] = df["label"].astype(int)


# df = df[["code", "Correct/incorrect"]].dropna()
# df = df.rename(columns={"Correct/incorrect": "label"})
# df["label"] = df["label"].astype(int)


dataset = Dataset.from_pandas(df)


In [16]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

def preprocess(examples):
    return tokenizer(examples["code"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(preprocess, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 3000/3000 [00:05<00:00, 503.95 examples/s]


In [17]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=1)


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:


training_args = TrainingArguments(
    output_dir="./codebert-correctness-gcj-exp1",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

trainer.train()


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.712500
20,0.727500
30,0.706800
40,0.697100
50,0.706100
60,0.693400
70,0.708100
80,0.684600
90,0.703400
100,0.665300


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=1125, training_loss=0.6904531461927625, metrics={'train_runtime': 4429.926, 'train_samples_per_second': 2.032, 'train_steps_per_second': 0.254, 'total_flos': 1183999749120000.0, 'train_loss': 0.6904531461927625, 'epoch': 3.0})

In [19]:
    # Save the model
model.save_pretrained("./codebert-correctness-gcj-exp1")
tokenizer.save_pretrained("./codebert-correctness-gcj-exp1")


('./codebert-correctness-gcj-exp1/tokenizer_config.json',
 './codebert-correctness-gcj-exp1/special_tokens_map.json',
 './codebert-correctness-gcj-exp1/vocab.json',
 './codebert-correctness-gcj-exp1/merges.txt',
 './codebert-correctness-gcj-exp1/added_tokens.json')